Script processing daily highprofit portfolios report.
This script:
- generates 99, 98....95th percentiles of profitability
- automatically prepares files: delete excess columns, add necessary ones (for ex. with calculated portfolio sums for yesteday), set proper file formatting
- adds data validation to field "Reason" (of abnormal profitability) - user can choose only values from dropdown list (standard reasons should've been used in Weekly Review of abnormal portfolio profitabilities)\

Pandas, os, openpyxl libraries used.

Скрипт, обрабатывающий ежедневный отчёт об аномальных доходностях.
Данный скрипт:\
    - формирует 99, 98...95-й перцентили доходности\
    - автоматически подготавливает файлы: убирает ненужные колонки, добавляет необходимые (например, со расчётными стоимостями портфелю за вчерашний день), настраивает форматирование файла\
    - добавляет проверку данных в поле с причиной аномальной доходности - пользователь может выбрать только из выпадающего списка (стандартизированные причины пригодились бы в дальнейшем для еженедельной сводки по аномальным доходностям)\
    - предусматривает логику: если в 99-м перцентиле более 15 портфелей, то в файлы с перцентилями записывается по 15 портфелей

In [1]:
import pandas as pd
import os

In [2]:
# 'Check a presence of daily highprofit reports in the same folder with script and press Enter'
a = input(prompt='Проверьте наличие ежедневных отчётов в папке со скриптом и нажмите Enter:')

os.listdir()
file_name_list = [file for file in os.listdir() if file.startswith('InvestHighProfitRep') & \
                                              file.endswith('.xlsx')]
# Define a function, generating excel-files
# Объявим функцию, формирующую excel-файлы
def generate_percentiles_df_to_xls(percentile_99, \
                                   percentile_98, \
                                   percentile_97, \
                                   percentile_96, \
                                   percentile_95, file_name):
    from openpyxl import Workbook
    from openpyxl.worksheet.datavalidation import DataValidation
    from openpyxl.styles import Font, Alignment
    from openpyxl.styles.borders import Border, Side
    from openpyxl import load_workbook

    numbers_of_perc = [99, 98, 97, 96, 95]

    for number_of_perc in numbers_of_perc:
        
        # Convert date to a proper format (without time) and profitability to percentage (round to 4 digits after the point)
        # Преобразуем дату в нужный формат (без времени) и доходность в процентную с округлением до 4-х знаков после запятой
        globals()[f"percentile_{number_of_perc}"].loc[:,'Дата'] = pd.to_datetime(globals()[f"percentile_{number_of_perc}"]['Дата']).dt.date
        globals()[f"percentile_{number_of_perc}"]['Доходность за день'] = globals()[f"percentile_{number_of_perc}"]['Доходность за день'].apply(lambda x: "{:.4%}".format(x).replace('.', ','))
        
        if number_of_perc == 99:
            percentile_99.to_excel(f"Result_{file_name}", index=False)        
            number_of_rows = percentile_99.shape[0]
            number_of_cols = percentile_99.shape[1]
            wb = load_workbook(filename = f"Result_{file_name}")

        else:
            globals()[f"percentile_{number_of_perc}"].to_excel(f"{number_of_perc}_{file_name}", index=False)
            number_of_rows = globals()[f"percentile_{number_of_perc}"].shape[0]
            number_of_cols = globals()[f"percentile_{number_of_perc}"].shape[1]
            wb = load_workbook(filename = f"{number_of_perc}_{file_name}")

        ws = wb.active
        ws.title = 'All'

        # Prepare border styles
        # Подготовим стиль границ ячеек, зададим позже
        border = Border(left=Side(style='thin'),
                right=Side(style='thin'),
                top=Side(style='thin'),
                bottom=Side(style='thin'))

        # Check data in column J (penultimate)        
        # Проверка данных в столбце J (предпоследний) 
        dv = DataValidation(type="list", formula1='"Пополнение брокерского счета,\
Вывод денежных средств,\
Депозитарный перевод ЦБ,\
Некорректная стоимость портфеля,\
Покупка/Продажа ВНБ,\
Заработок на торговле,\
Погашение облигаций/НКД,\
Участие в Pre-IPO/IPO,\
Корректировка налога"', allow_blank=False, showDropDown=False)
        dv.errorTitle = 'Ошибка (Причина аномальной доходности)'
        dv.error = 'Выберите причину аномальной доходности из выпадающего списка'
        dv.prompt = 'Введите причину аномальной доходности'
        ws.add_data_validation(dv)
        dv.add(f"{chr(64 + number_of_cols - 1)}2:{chr(64 + number_of_cols - 1)}{number_of_rows + 1}")

        # Add line breakes in columns with long titles (bestFit works incorrect)
        # Добавим перенос текста в столбцах с длинными заголовками (bestFit работает некорректно)
        ws['D1'].alignment = Alignment(wrap_text = True, horizontal='center', vertical='top')
        ws['E1'].alignment = Alignment(wrap_text = True, horizontal='center', vertical='top')
        ws['F1'].alignment = Alignment(wrap_text = True, horizontal='center', vertical='top')
        ws['H1'].alignment = Alignment(wrap_text = True, horizontal='center', vertical='top')
        ws['I1'].alignment = Alignment(wrap_text = True, horizontal='center', vertical='top')

        for cols in range(1, number_of_cols + 1): 
            ws.column_dimensions[f"{chr(64 + cols)}"].bestFit = True
            for rows in range(1, number_of_rows + 2): #не забываем про строку с заголовками, поэтому "+2"
                ws[f"{chr(64 + cols)}{rows}"].border = border

        # Set (manually) columns where bestFit doesn't set width as it should 
        # Вручную настроим колонки, в которых bestFit не лучшим образом задаёт ширину
        ws.column_dimensions['H'].width = 20
        ws.column_dimensions['I'].width = 20
        ws.column_dimensions['J'].width = 35
        ws.column_dimensions['K'].width = 40

        if number_of_perc == 99:
            wb.save(f"Result_{file_name}")
        else:
            wb.save(f"{number_of_perc}_{file_name}")
    return 'Percentiles generated successfully!'

for file_name in file_name_list:
    df = pd.read_excel(f"./{file_name}")
    
    # Delete excess columns and sort by descending profitability
    # Удалим лишние колонки и отсортируем по убыванию доходности
    df = df.drop(columns=['Доходность за 7 дней', 'Доходность за 30 дней']).sort_values('Доходность за день', ascending=False)
    
    # Keep non-negative profitabilities only
    # Оставим только неотрицательные доходности
    df = df.loc[df["Доходность за день"] >= 0].reset_index(drop=True)
    
    # Add columns with cost of portfolio yesterday and today + columns for "reason" and "description"
    # Добавим колонки стоимостей портфеля вчера и сегодня, а также столбцы для причины и описания
    df['Стоимость портфеля DWH на предыдущий день'] = round(df['Стоимость портфеля'] / (1 + df['Доходность за день']), 2)
    df['Стоимость портфеля расчетная'] = round(df['Стоимость портфеля'], 2)
    df['Причина'] = ''
    df['Описание'] = ''
    
    # 99 percentile
    df['Доходность за день'].quantile(0.99)
    percentile_99 = df.loc[df['Доходность за день'] >= df['Доходность за день'].quantile(0.99)]

    # If 99th percentile has more than 15 lines, we put in every percentile 15 inv.accounts with abnormal profitability
    # Учтём логику: если в 99-м перцентиле больше 15 строк, в каждый перцентиль кладём по 15 БС с аномальной доходностью
    if percentile_99.shape[0] > 15:
        percentile_99 = df[0:15]
        percentile_98 = df[15:30]
        percentile_97 = df[30:45]
        percentile_96 = df[45:60]
        percentile_95 = df[60:75]
    else:
        percentile_98 = df.loc[df['Доходность за день'] >= df['Доходность за день'].quantile(0.98)]\
          .loc[df['Доходность за день'] < df['Доходность за день'].quantile(0.99)]
        percentile_97 = df.loc[df['Доходность за день'] >= df['Доходность за день'].quantile(0.97)]\
          .loc[df['Доходность за день'] < df['Доходность за день'].quantile(0.98)]
        percentile_96 = df.loc[df['Доходность за день'] >= df['Доходность за день'].quantile(0.96)]\
          .loc[df['Доходность за день'] < df['Доходность за день'].quantile(0.97)]
        percentile_95 = df.loc[df['Доходность за день'] >= df['Доходность за день'].quantile(0.95)]\
          .loc[df['Доходность за день'] < df['Доходность за день'].quantile(0.96)]
        
    pd.options.mode.chained_assignment = None
    percentile_99.loc[:,'Дата'] = pd.to_datetime(percentile_99['Дата']).dt.date
    generate_percentiles_df_to_xls(percentile_99,percentile_98,percentile_97,percentile_96,percentile_95, file_name)

Проверьте наличие ежедневных отчётов в папке со скриптом и нажмите Enter:


C:\Users\i.baryshnikov\AppData\Local\Temp\ipykernel_12616\2825161699.py:123: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  percentile_99.loc[:,'Дата'] = pd.to_datetime(percentile_99['Дата']).dt.date
C:\Users\i.baryshnikov\AppData\Local\Temp\ipykernel_12616\2825161699.py:22: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  globals()[f"percentile_{number_of_perc}"].loc[:,'Дата'] = pd.to_datetime(globals()[f"percentile_{number_of_perc}"]['Дата']).dt.date
C:\Users\i.baryshnikov\AppData\Local\Temp\ipykernel_12616\2825161699.py:22: DeprecationWarnin